- Groq is a company that develops high-performance hardware and software solutions for AI inference. Their core product, the LPU™   Inference Engine, is designed to accelerate AI tasks, offering significant speed improvements. It provides superior compute power, enabling faster processing of AI models. The platform is optimized for energy efficiency, reducing operational costs. Overall, Groq aims to enhance AI performance across various applications, including machine learning and deep learning workloads.
- https://console.groq.com/playground

In [2]:
import os
from dotenv import load_dotenv
import openai
load_dotenv()

openai.api_key=os.getenv('OPENAI_API_KEY')

groq_api_key=os.getenv('GROQ_API_KEY')

In [4]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

groq_model = ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)
openai_model = ChatOpenAI(model="gpt-4o")


In [9]:
from langchain_core.messages import SystemMessage,AIMessage,HumanMessage

message = [
        SystemMessage("Translate from English to Hindi"),
        HumanMessage(content="Hello how are you")
]

response1 = groq_model.invoke(message)
response2 = openai_model.invoke(message)

print(response1)
print("\n"*2)
print(response2)


content='नमस्ते, आप कैसे हैं? \n\n(Namaste, aap kaise hain?) \n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 18, 'total_tokens': 42, 'completion_time': 0.043636364, 'prompt_time': 7.593e-05, 'queue_time': 0.020824519000000003, 'total_time': 0.043712294}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-5cd4fc59-ba81-4f36-9ab3-ffac1c4ac544-0' usage_metadata={'input_tokens': 18, 'output_tokens': 24, 'total_tokens': 42}



content='नमस्ते, आप कैसे हैं?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 20, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'fi

In [10]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

parser.invoke(response1)

'नमस्ते, आप कैसे हैं? \n\n(Namaste, aap kaise hain?) \n'

In [11]:
#using LCEL we can chain the components

In [15]:
chain1= groq_model|parser
print(chain1.invoke(message))

chain2= openai_model|parser
print(chain2.invoke(message))


नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) 

नमस्ते, आप कैसे हैं?


In [18]:
#another efficient method is using prompt templates

from langchain_core.prompts import ChatPromptTemplate

generic_prompt = "Translate into following {language}"

prompt = ChatPromptTemplate.from_messages([("system",generic_prompt),("user","{text}")])

prompt.invoke({"language":"Hindi","text":"Hello this is shanmukh"})

chain3 = prompt | groq_model | parser

chain3.invoke({"language":"Hindi","text":"Hello this is shanmukh"})



'नमस्ते, मैं शंमुख हूँ। \n\n(Namaste, main Shanmukha hoon.)\n'